In [1]:
import pandas as pd
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

In [2]:
os.getcwd()

'C:\\Users\\claud\\Documents\\GitHub\\vivienda_cdmx\\predial'

In [3]:
def busca_adeudos(cuenta):
    url="https://ovica.finanzas.cdmx.gob.mx/ovica-backend/public/api/v1/adeudos/vencido/"+cuenta
    r=requests.get(url)
    try:
        data=r.json()
        #Obtener ejercicios de adeudos, periodos y montos
        ejercicio=[]
        periodo=[]
        monto=[]
        
        
        for i in range(len(data)):
            ejercicio.append(data[i]['ANO'])
            periodo.append(data[i]['PERIODO'])
            monto.append(data[i]['SALDO'])
            
        df_ad=pd.DataFrame({
            'cuenta_adeudo': [cuenta] * len(ejercicio),
            'cuenta_shp': [cuenta[:8]] * len(ejercicio),
            'anio':ejercicio, 'bimestre':periodo, 'monto':monto})
    #Variables a numerico
        df_ad['anio']=pd.to_numeric(df_ad['anio'])
        df_ad['bimestre']=pd.to_numeric(df_ad['bimestre'])
        df_ad['monto']=pd.to_numeric(df_ad['monto'])
        
    except:
        df_ad = pd.DataFrame()
    return df_ad
        

In [4]:
# # Function to fetch data for a single cuenta
# def fetch_adeudo(cuenta):
#     try:
#         return busca_adeudos(cuenta)  # Calls your function
#     except Exception as e:
#         print(f"Error con la cuenta {cuenta}: {e}")
#         return pd.DataFrame()  # Return an empty DataFrame if there's an error

# # Parallel processing function
# def parallel_fetch(cuentas, max_workers=10):
#     adeudo_tot = pd.DataFrame()
    
#     with ThreadPoolExecutor(max_workers=max_workers) as executor:
#         # Submit all requests
#         futures = {executor.submit(fetch_adeudo, cuenta): cuenta for cuenta in cuentas}
        
#         # Use tqdm for progress tracking
#         for future in tqdm(as_completed(futures), total=len(futures)):
#             try:
#                 result = future.result()
#                 if not result.empty:  # Concatenate non-empty DataFrames
#                     adeudo_tot = pd.concat([adeudo_tot, result], ignore_index=True)
#             except Exception as e:
#                 print(f"Ocurrió un error: {e}")
    
#     return adeudo_tot


In [5]:
# Function to fetch data for a single cuenta
def fetch_adeudo(cuenta):
    try:
        return busca_adeudos(cuenta)  # Calls your function
    except Exception as e:
        return None  # Return None for failed requests


def parallel_fetch(cuentas, max_workers=10):
    adeudo_tot = pd.DataFrame()
    failed_cuentas = []  # List to store failed cuentas

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all requests
        futures = {executor.submit(fetch_adeudo, cuenta): cuenta for cuenta in cuentas}
        
        # Create a tqdm progress bar for the total number of requests
        with tqdm(total=len(futures), desc="Procesando requests", ncols=80) as pbar:
            for future in as_completed(futures):
                cuenta = futures[future]  # Get the corresponding cuenta
                try:
                    result = future.result()
                    if result is None:
                        failed_cuentas.append(cuenta)  # Save cuenta for retry
                    elif not result.empty:  # Concatenate non-empty DataFrames
                        adeudo_tot = pd.concat([adeudo_tot, result], ignore_index=True)
                except Exception as e:
                    failed_cuentas.append(cuenta)  # Save cuenta for retry if an error occurs
                
                # Update the progress bar after each completed request
                pbar.update(1)
    
    return adeudo_tot, failed_cuentas

In [6]:
#Leer cuentas únicas
padron_predial=pd.read_csv('cuentas_unicas/cuentas_unicas_1500000.txt', header=None).rename(columns={0:'cuenta_adeudo'})
print(len(padron_predial))
#Transformar a lista
lote=padron_predial['cuenta_adeudo'].tolist()

100000


In [7]:
path = "cuentas_fallidas"
files=os.listdir(path)
files

['failed_cuentas_0.txt',
 'failed_cuentas_100000.txt',
 'failed_cuentas_1000000.txt',
 'failed_cuentas_1100000.txt',
 'failed_cuentas_1200000.txt',
 'failed_cuentas_1300000.txt',
 'failed_cuentas_1400000.txt',
 'failed_cuentas_1500000.txt',
 'failed_cuentas_1600000.txt',
 'failed_cuentas_1700000.txt',
 'failed_cuentas_1900000.txt',
 'failed_cuentas_200000.txt',
 'failed_cuentas_2000000.txt',
 'failed_cuentas_2100000.txt',
 'failed_cuentas_2200000.txt',
 'failed_cuentas_300000.txt',
 'failed_cuentas_400000.txt',
 'failed_cuentas_500000.txt',
 'failed_cuentas_600000.txt',
 'failed_cuentas_700000.txt',
 'failed_cuentas_800000.txt',
 'failed_cuentas_900000.txt',
 'failed_cuentas_partial.txt']

In [8]:

# Initialize an empty DataFrame
df = pd.DataFrame()


for file in files:
    file_path = os.path.join(path, file)
    if os.path.getsize(file_path) > 0:  # Check if the file is not empty
        try:
            # Load the file as a single column without headers
            temp_df = pd.read_csv(file_path, header=None, names=['cuenta_adeudo'], low_memory=False)
            if not temp_df.empty:  # Check if the DataFrame is not empty
                df = pd.concat([df, temp_df], ignore_index=True)
            else:
                print(f"Skipping empty file: {file}")
        except pd.errors.EmptyDataError:
            print(f"Skipping invalid file: {file}")
    else:
        print(f"Skipping empty file: {file}")




Skipping empty file: failed_cuentas_partial.txt


In [9]:
#Largo de cuentas fallidas a 12 caracteres
df['cuenta_adeudo'] = df['cuenta_adeudo'].astype(str).str.zfill(12)
#Transformar a lista
lote=df['cuenta_adeudo'].tolist()
print(len(lote))

200063


In [13]:
lote=["016064040039"]

### Buscar adeudos

In [14]:
adeudo_tot,failed_cuentas = parallel_fetch(lote, max_workers=10)

Procesando requests: 100%|████████████████████████| 1/1 [00:00<00:00,  2.45it/s]


In [15]:
print("Total de registros: ", len(adeudo_tot))

Total de registros:  41


In [16]:
adeudo_tot.to_csv('adeudos/adeudos_finales_finales.csv', index=False)

with open('cuentas_fallidas/failed_cuentas_finales_finales  .txt', 'w') as f:
    # Convert all items in failed_cuentas to strings before joining
    f.write('\n'.join(map(str, failed_cuentas)))